# Interactive type checking

In [a previous notebook about type checking](2018-06-20-Interactive-type-checking-with-datashape.ipynb) we successfully explored interactive type checking with numerical data using [`blaze/datashape`](https://datashape.readthedocs.io/).  In the demonstrations:

* Too much focus was placed on mapping generic types across input and output types.
* There was no consideration of actual Python types.
* _It was incomplete._

There are increasingly frequent discussions around typing [numpy](https://twitter.com/dabeaz/status/1010112644819439616) [arrays](https://docs.google.com/document/d/1vpMse4c6DrWH5rq2tQSx3qwP_m_0lyn-Ij4WHqQqRHY/mobilebasic#) or the [stub files](https://github.com/numpy/numpy-stubs).

In [`dabeaz/typemap`](https://github.com/dabeaz/typemap#faq) there is a comment cautioning:

> However, keep in mind that good type annotations should be hard to type.


Mypy is a static type checker, it does not know anything about run time events.  There are cool tools like [`dropbox/pyannotate`](http://mypy-lang.blogspot.com/2017/11/dropbox-releases-pyannotate-auto.html) and [`instagram/monkeytype`](https://github.com/Instagram/MonkeyType) that will infer types from run times events.

In addition to numpy types, can we __interactively__ validate complex numerical types, json and xml schemas, and other conditional logic?

> Static typing is valuable to large applications across many members.  However, for small projects, typing can be a distraction while interactively computing.  Can we have both though?

In [1]:
    from datashape import dshape, DataShape, validate, TypeVar
    from inspect import getfullargspec
    from functools import wraps
    from toolz.curried import excepts, flip
    from IPython import get
    import numpy as np

## Interactive typing use cases.

Consider two common operations like the `dot` production and `mat`rix`mul`tiplication.  In both of these operations, a user expects an output shape based on the inputs and operations.

In [2]:
    def dot(
        x: dshape("N * float64"),
        y: dshape("N * float64"),
      ) -> float:
        return sum(xᵢ * yᵢ for xᵢ, yᵢ in zip(x, y))

### [`datashape`](https://datashape.readthedocs.io/) provides a grammar for expressing complex multi-dimensional numerical types.  `dshape` is used to defined the expected input and output shapes.

In [40]:
    def matmul(
        x: dshape("J * K * float64"),
        y: dshape("K * L * float64"),
    ) -> dshape("J * L * float64"):
        return x @ y

# Generics

[Generics](https://mypy.readthedocs.io/en/latest/generics.html) are parameterized types, `datashape` expresses them as `TypeVar`.  `TypeVar` are created by `datashape.dshape` which converts the `datashape` text format to a `TypeVar`.  For example, the statement below describes a $[MxN]$ matrix composed of 8-bit integers.



In [4]:
    sample = dshape("M*N*int8")

The `datashape.validate` functions can be used to check the consistency.

In [5]:
    assert validate(sample, np.ones((10,10), dtype='int8')) is None

# Discovery

`datashape.discover` discovers the datashape from python and numpy objects.  The discovery system is extensible using [multipledispatch](https://github.com/mrocklin/multipledispatch)
methods.  Our discover method needs to be patched for finding string types better.

In [6]:
    def discover(object): 
        from datashape import discover
        return dshape('string') if isinstance(object, str) else discover(object)

# Checking arguments.

For a set of annotations, the named arguments values should validate againsted Python or Datashape types.

In [7]:
    def check_args(annotations, names, args, generic=None):
        """interactively check annotations."""
        if generic is None:
            generic = {}
        for name, value in zip(names, args):
            annotation = annotations.get(name)
            if isinstance(annotation, type):
                if not isinstance(value, annotation):
                    raise IncompatibleDataTypes(
                        f"""{name} expects {annotation}, but recieved {type(value)}"""
                    )
            elif isinstance(annotation, DataShape):
                annotation = update_generic_annotation(generic, annotation, value)
                if not validate(annotation, value):
                    raise IncompatibleDataTypes(
                        f"""{name} expects {annotation}, but recieved {discover(value)}"""
                    )
            elif callable(annotation):
                """Just for kicks let's put a callable here."""
                if not annotation(value):
                    raise IncompatibleDataTypes(f"""The {name} value does not satisfy {annotation}.""")
        return generic

In [8]:
    class IncompatibleDataTypes(BaseException): ...

`update_generic_annotation` will catch the first the shape or type of the first instance it encounters.  Afterwards, that type is mapped to every other instance of the same `TypeVar`

In [9]:
    def update_generic_annotation(generic, annotation, value):
        from datashape.coretypes import CType
        if isinstance(annotation, DataShape):
            if annotation.composite:
                discovered, new_parameters = discover(value), []
                parameters, actuals = annotation.info(), discovered.info()
                
                # The ctype condition is for standard types 'int', 'string' dshapes.
                if actuals[0] is CType:  actuals = actuals[0], [actuals[1][0]]
                if discovered[0] is CType: parameters = parameters[0], [parameters[1][0]]
                    
                for parameter, actual in zip(parameters[1], actuals[1]):
                    if isinstance(parameter, TypeVar):
                        if parameter not in generic:
                            generic[parameter] = actual
                        parameter = generic[parameter]
                    new_parameters.append(parameter)

                if len(parameters) != len(actuals):
                    print(parameters, actuals)
                    return annotation

                annotation = DataShape(*new_parameters)
        return annotation

> This approach does not allow for global generics to be defined.

## The `itypecheck` decorator

For this example, the functions are tested using a decorator, but this eventually could be closer to the runtime events using the `sys.profile`.

In [41]:
    def itypecheck(callable):
        spec = getfullargspec(callable)

        @wraps(callable)
        def check(*args, **kwargs):
            generic = check_args(spec.annotations, spec.args, args)
            result = callable(*args)
            check_args(spec.annotations, ["return"], [result], generic)
            return result

        return check


## Basic interactive Python type checking

> Sometimes we will test for incompatible types.  These cases are flagged by `__exception__`

In [42]:
    __exception__ = flip(excepts(IncompatibleDataTypes))(print)

In [44]:
    @itypecheck
    def to_string(x: int) -> str: 
        return x

The type checker catches a failure in this function because it returns the wrong type

In [45]:
    __exception__(to_string)(20)

return expects <class 'str'>, but recieved <class 'int'>


We can fix this by returning a string.

In [46]:
    @itypecheck
    def to_string(x: int) -> str: return f"{x}"

    to_string(20)

'20'

The datashape equivalent

In [47]:
    @itypecheck
    def to_string(x: dshape('int')) -> dshape('string'): 
        return f"{x}"

    to_string(20)

'20'

### Simple generic types

In [48]:
    @itypecheck
    def f_(x: dshape("2*V"))->dshape('V'):
        return str(x[0])

    __exception__(f_)([10, 20])

return expects int64, but recieved string


In [49]:
    @itypecheck
    def f(x: dshape("2*V"))->dshape('V'):
        return x[0]

    def _generic_type_checking(): f([10, 20])

10

## Numerical Examples.

The real goal is type check generically sized numerical types.  Our original functions work without type checking.

In [52]:
    dot(np.array([10]), np.array([20], dtype=int))

200

What happens when the `datashape` interactive type checking is added?

### Dot Product

In [19]:
    _dot = itypecheck(dot)

In [53]:
    __exception__(_dot)(np.array([10]), np.array([20], dtype=int))

x expects 1 * float64, but recieved 1 * int64


In [54]:
    itypecheck(dot)(np.array([10.]), np.array([20.]))

200.0

### Matrix Multiplication

In [55]:
    _matmul = itypecheck(matmul)

In [56]:
    _matmul(np.ones((5, 3)), np.ones((3, 3)))

array([[ 3.,  3.,  3.],
       [ 3.,  3.,  3.],
       [ 3.,  3.,  3.],
       [ 3.,  3.,  3.],
       [ 3.,  3.,  3.]])

`itypecheck` will catch mismatched types ...

In [57]:
    __exception__(_matmul)(np.ones((5, 3)), np.ones((3, 3), dtype='int'))

y expects 3 * 3 * float64, but recieved 3 * 3 * int64


and sizes ...

In [58]:
    __exception__(_matmul)(np.ones((5, 3)), np.ones((5, 3)))

y expects 3 * 3 * float64, but recieved 5 * 3 * float64


That provide useful tracebacks to the author.

### Did you know numpy did this?

Consider the interesting case of numpy automatically converting int32 values into int64 in the function above.

In [59]:
    @itypecheck
    def dot_multiply(x: dshape('M*T'), y: dshape('M*T')) -> dshape('M*int'):
        return np.multiply(x, y)

In [61]:
    __exception__(dot_multiply)(np.arange(10), np.arange(10))

return expects 10 * int32, but recieved 10 * int64


### Answering a [twitter question](https://twitter.com/dabeaz/status/1010112644819439616)

> Suppose I write a function:

>     def vec3(x:float, y:float, z:float):
>          return numpy.array((x,y,z))

> I now use this to make "instances" of 3D vectors, which get passed around and used throughout my code.
> How would I type hint all of that?

In [63]:
@itypecheck
def vec3(x:float, y:float, z:float)->dshape('3*float64'):
     return np.array((x,y,z))

* This works

In [64]:
    vec3(1., 2., 3.)

array([ 1.,  2.,  3.])

* This doesn't work because the input type is wrong.

In [65]:
    __exception__(vec3)(1., 2., 3)

z expects <class 'float'>, but recieved <class 'int'>


Clearly some magical things happen in numerical computing and type checking could catch some unexpected behaviors.

## Discussions

* What is the best way to overload the types?
* Interactive type checking (type checking with a run time) could check other conditions like json schema, [marshmallow](https://github.com/marshmallow-code/marshmallow), traitlets, symmetry.
* Should data output payloads be checked?  What is matplotlib inline should not be on?
* How can parity be acheived with static checkers ([`python/mypy`](https://github.com/python/mypy), [`facebook/pyre-check`](https://pyre-check.org/))to use subsets of interactive type checking.
* Type checking numpy supersets like astropy and dataframes.
* Is there a more pythonic API to representing datashape?  Would this allow globally scoped generics.  An API closer to what is suggested in [Ideas for array shape typing in Python](https://docs.google.com/document/d/1vpMse4c6DrWH5rq2tQSx3qwP_m_0lyn-Ij4WHqQqRHY/mobilebasic#)
* Can this be part of the profiler automatically?

In [1]:
    from . import disqus